In [1]:
!pip install --upgrade keras

Requirement already up-to-date: keras in /usr/local/lib/python3.7/dist-packages (2.4.3)


In [2]:
import csv
import nltk
import pandas as pd
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout, Bidirectional
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
class SentimentNLP():
  def __init__(self, 
               sequence_length=2000, 
               embed_dim=128, 
               batch_size=32, 
               epochs=200,
               num_words=1000,
               oov_token="<UNK>",
               pad_type="post",
               trunc_type="post"):
    


    self.sequence_length = sequence_length
    self.embed_dim = embed_dim
    self.batch_size = batch_size
    self.epochs = epochs

    self.num_words = num_words
    self.oov_token = oov_token
    self.pad_type = pad_type
    self.trunc_type = trunc_type


    print("Loading Data.......")
    df = self.load_data()


    print("Formatting Data......")
    sentences = df["sentences"]

    sentences = self.lower_case(sentences)

    sentences_sequences, word_index, maxlen, sentences_padded = self.tokenize_data(sentences)


    print("Data Stats......")
    print("Word index:\n", word_index)
    print("\nTraining sequences:\n", sentences_sequences)
    print("\nPadded training sequences:\n", sentences_padded)

    print("Splitting Data....")
    X_train, X_test, y_train, y_test = self.get_data(sentences_padded)

    print(f"X_train:\n {X_train}")
    print(f"X_test:\n {X_test}")
    print(f"y_train:\n {y_train}")
    print(f"y_test:\n {y_test}")
    


    print("Building Model.......")
    model = self.build_model(X_train)
    print("Built Model,")


    print("Training Model.......")
    history = self.train_dataset(model, X_train, y_train, X_test, y_test)

    print("Model Trained.")

    # print("Saving Plots.........")
    # self.plot_acc(history)
    # self.plot_loss(history)


  def load_data(self):
    df = pd.read_csv("data2.csv")
    return df

  def lower_case(self, text_list):
    lower_phrases = []
    for phrase in text_list:
      lower_phrases.append(phrase.lower())
    
    return lower_phrases
  
  def tokenize_data(self, text_list, num_words=1000, oov_token="<UNK>", pad_type="post", trunc_type="post"):
    tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
    tokenizer.fit_on_texts(text_list)
    word_index = tokenizer.word_index
    train_sequences = tokenizer.texts_to_sequences(text_list)
    
    print("here")
    
    with open('word_tokenizer.pickle', 'wb') as f:
      pickle.dump(tokenizer, f)

    maxlen = max([len(x) for x in train_sequences])
    
    train_padded = pad_sequences(train_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)



    return train_sequences, word_index, maxlen, train_padded

  
  def get_data(self, train_data):
    df = self.load_data()
    train_classes = pd.get_dummies(df["class_names"])
    # train_classes = train_classes.iloc[:, :-1]

                                   
    X, y = train_data, train_classes
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    X_train = np.asarray(X_train)
    X_test = np.asarray(X_test)
    y_train = np.asarray(y_train)
    y_test = np.asarray(y_test)

    return X_train, X_test, y_train, y_test

  def build_model(self, X, sequence_length=2000, embed_dim=128):
    model = Sequential()
    model.add(Embedding(sequence_length, embed_dim, input_length=X.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(Bidirectional(CuDNNLSTM(196)))
    model.add(Dropout(0.2))
    model.add(Dense(2, activation="sigmoid"))
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model 

  def train_dataset(self, model, X_train, y_train, X_test, y_test, batch_size=32):      
      history = model.fit(X_train, y_train, epochs=37, batch_size=batch_size, verbose=2)

      model.save("word_analysis.h5")

      score, acc = model.evaluate(X_test, y_test, verbose=2, batch_size=batch_size)

      print("score: %.2f" % (score))
      print("acc: %.2f" % (acc))

      return history

  # def plot_acc(self, history):
  #   plt.plot(history.history['accuracy'])
  #   plt.title('model accuracy')
  #   plt.ylabel('accuracy')
  #   plt.xlabel('epoch')
  #   plt.legend(['train'], loc='upper left')
  #   plt.savefig('graphs/acc.png')


  # def plot_loss(self, history):
  #   plt.plot(history.history['loss'])
  #   plt.title('model loss')
  #   plt.ylabel('loss')
  #   plt.xlabel('epoch')
  #   plt.legend(['train'], loc='upper left')
  #   plt.savefig('graphs/loss.png')

In [9]:
sentimentNLP = SentimentNLP()

Loading Data.......
Formatting Data......
here
Data Stats......
Word index:
 {'<UNK>': 1, 'well': 2, 'ill': 3, 'self': 4, 'anti': 5, 'free': 6, 'down': 7, 'hard': 8, 'up': 9, 'over': 10, 'known': 11, 'low': 12, 'best': 13, 'priced': 14, 'top': 15, 'balanced': 16, 'better': 17, 'cost': 18, 'counter': 19, 'dead': 20, 'cheap': 21, 'fast': 22, 'rich': 23, 'first': 24, 'class': 25, 'friendly': 26, 'god': 27, 'high': 28, 'long': 29, 'risk': 30, 'non': 31, 'problem': 32, 'time': 33, 'back': 34, 'break': 35, 'out': 36, 'fat': 37, 'all': 38, 'awe': 39, 'clear': 40, 'confidence': 41, 'effective': 42, 'saving': 43, 'attack': 44, 'attacks': 45, 'crisp': 46, 'cure': 47, 'defeating': 48, 'dirt': 49, 'easy': 50, 'educated': 51, 'efficient': 52, 'eminent': 53, 'energy': 54, 'envious': 55, 'enviously': 56, 'enviousness': 57, 'err': 58, 'eye': 59, 'famous': 60, 'growing': 61, 'fastest': 62, 'favored': 63, 'fine': 64, 'happy': 65, 'quality': 66, 'spirited': 67, 'honored': 68, 'issue': 69, 'jaw': 70, 'hea